# Medición de Tiempos: Python, Numpy vs GPU (TensorFlow)

* En este notebook vamos a medir los tiempos de ejecución en la multiplicción de 2 matrices usando:

    - **Python - CPU**: implementación "tradicional" para la multiplicación de 2 Matrices.
    - **Numpy - CPU**: Llamando al método *.dot()* para la multiplicación de matrices.
    - **TensorFlow - GPU**: Definidos dos tensores y los multiplicamos con el método *.multiply()*. Para esta ejecución se hará uso de la GPU "Nvidia GeForce RTX 2060" con:
        + Drivers instalados
        + CUDA (versión 10.1)
        + CUDNN (versión 7)
        + TensorFlow (versión 2.3.0)
        
        
<hr>


### Definimos 2 matrices aleatorias

In [1]:
import numpy as np

# Tamaño de la matriz SIZExSIZE
SIZE = 500
a = np.random.rand(SIZE, SIZE)
b = np.random.rand(SIZE, SIZE)


## 1.- Python - CPU

In [2]:
%%timeit
result = np.zeros((SIZE, SIZE))
for i in range(SIZE):
    for j in range(SIZE):
        for k in range(SIZE):
            result[i,j] += a[i,k] * b[k,j]

1min 3s ± 763 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2.- Numpy - CPU

In [3]:
%%timeit
x = np.dot(a,b)

1.79 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 3.- TensorFlow - GPU

* En primer lugar:
    - Limitamos el uso de la memoria de la GPU (recomendable para que no "casquen" nuestros programas)
    - Mostramos la información relativa a la GPU y las versiones de CUDA y CUDNN instaladas

In [4]:
import tensorflow as tf

# Limitación la memoria de la GPU
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.6
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# Permitir crecimiento de la memoria
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')


print('#### INFORMACIÓN ####')
print('  Versión de TensorFlow: {}'.format(tf.__version__))
print('  GPU: {}'.format([(x.name, x.physical_device_desc)
                          for x in tf.python.client.device_lib.list_local_devices() if x.device_type == 'GPU']))
print('  Versión Cuda  -> {}'.format(tf.sysconfig.get_build_info()['cuda_version']))
print('  Versión Cudnn -> {}'.format(tf.sysconfig.get_build_info()['cudnn_version']))

#### INFORMACIÓN ####
  Versión de TensorFlow: 2.3.0
  GPU: [('/device:GPU:0', 'device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5')]
  Versión Cuda  -> 64_101
  Versión Cudnn -> 64_7


* Pasamos las matrices a tensores

In [5]:
# Creamos los tensores
ta = tf.convert_to_tensor(a)
tb = tf.convert_to_tensor(b)


* Multiplicamos los tensores (con GPU)

In [6]:
%%timeit
result = tf.math.multiply(ta,tb)


28.3 µs ± 36.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
